Workflow:
0.libraries & device-agnostic
1.data acqusition
2.data preparation
3. transforming data
4. loading data with ImageFolder
5. loading image data with a custom Dataset
6.data augmentation
7. Model 0: TinyVGG without data augmmentation
8. explore loss curves
9.Model 1 TinyVGG with data augmentation
10. model results comparison
11. prediction on a custom image



In [2]:
import torch
from torch import nn

1. Data acqusition

In [ ]:
import requests
import zipfile
from pathlib import Path
#setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
#check image folder existence, if not create one&download&prepare it
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    #download pizza, steak,sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak & sushi data...")
        f.write(request.content)
    #unzip data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...<<don't start feeling hungry though:)")
        zip_ref.extractall(image_path)


2. Data Preparation

In [ ]:
#inspect contents of the data dir
import os 
def walk_through_dir(dir_path):
for dirpath.dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)}.directories and {len(filenames)}images in '{dirpath}.")


In [ ]:
#setting up train and test paths
train_dir = image_path / "train"
test_dir = image_path / "test"
train_dir, test_dir

In [ ]:
#2.1 Visualize an image
import random
from PIL import Image
random.seed(42)
#1.getting all image paths
image_path_list = list(image_path.glob("*/*/*.jpg")) #* means "any combination"
#2. get randomimage path
random_image_path = random.choice(image_path_list)
#3.get the name of the dir where image is stored
image_class = random_image_path.parents.stem
#4. Open image
img = Image.open(random_image_path)
#5. metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

3. Transforming data


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, disable_beta_transforms_warning